In [1]:
from IPython.core.magic import register_cell_magic, register_line_magic
from io import StringIO
from logica.common import logica_lib
import tempfile
import sqlite3
import pandas as pd
import numpy as np
import duckdb
import logica
import sys

In [2]:
! logica help

Usage:
  logica <l file> <command> <predicate name> [flags]
  Commands are:
    print: prints the StandardSQL query for the predicate.
    run: runs the StandardSQL query on BigQuery with pretty output.
    run_to_csv: runs the query on BigQuery with csv output.


Example:
  python3 logica.py - run GoodIdea <<<' GoodIdea(snack: "carrots")'


In [3]:
! pip show logica

Name: logica
Version: 1.3.14159265358
Summary: Logica language.
Home-page: https://github.com/evgskv/logica
Author: Evgeny Skvortsov
Author-email: logica@evgeny.ninja
License: UNKNOWN
Location: /Users/duncanw/repos/wdduncan/onto-logica/.venv/lib/python3.11/site-packages
Requires: 
Required-by: 


---

## Simple command line logica program.

In [4]:
! logica - print Greet <<<'Greet("howdy")'

SELECT
  "howdy" AS col0;


Attaching a local semsql sqlite database and pull out `OHD` row from `prefix` table.

In [5]:
! logica - run Prefix <<<'\
@Engine("sqlite"); \
@AttachDatabase("db", "../data/dental-material.db"); \
Prefix(..r) :- prefix(..r), r.prefix == "OHD"; '

+--------+-------------------------------------+
| prefix | base                                |
+--------+-------------------------------------+
| OHD    | http://purl.obolibrary.org/obo/OHD_ |
+--------+-------------------------------------+


---

## Execute logica in python.

Call logica program dynamically using `logica_lib.RunPredicateFromString`.

In [6]:
statement = """
@Engine("sqlite");

Test(y: value) :-
  value == x * x + 2 * x + 5,
  x == 10;
"""

In [7]:
logica_lib.RunPredicateFromString(statement, 'Test')

,y
0,125


---

## Working with sqlite database.

Connect to semsql sqlite database dynamically/programatically pull out sdo prefix row.

In [8]:
cn = sqlite3.connect('../data/dental-material.db')

In [9]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

Prefix(..r) :- prefix(..r), r.prefix == "OHD"; 

""", 'Prefix', cn)

,prefix,base
0,OHD,http://purl.obolibrary.org/obo/OHD_


Find class in ontology with label "ceramic dental restoration material".

In [10]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

Material(..r) :- 
    statements(..r), 
    r.predicate == "rdfs:label", 
    r.value == "ceramic dental restoration material"; 

""", 'Material', cn)

,stanza,subject,predicate,object,value,datatype,language
0,OHD:0000135,OHD:0000135,rdfs:label,None,ceramic dental restoration material,None,None


Find materials is a types of `ceramic dental restoration material`.

In [11]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

Material(subject:, predicate:, object:) :- 
    statements(subject:, predicate:, object:), 
    predicate == "rdfs:subClassOf", 
    object == "OHD:0000135"; 

""", 'Material', cn)

,subject,predicate,object
0,OHD:0001032,rdfs:subClassOf,OHD:0000135
1,OHD:0001033,rdfs:subClassOf,OHD:0000135
2,OHD:0001047,rdfs:subClassOf,OHD:0000135


Return labels for the subject and object of `ceramic dental restoration material` classes.  
To do this, define a `Label` predicate to find the value of `rdfs:label`.

In [12]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

# function to return the label of a class
Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);
    
Material(subject_label:, predicate:, object_label:) :- 
    statements(subject:, predicate:, object:), 
    predicate == "rdfs:subClassOf", 
    object == "OHD:0000135",
    subject_label == Label(subject),
    object_label == Label(object);

""", 'Material', cn)

,subject_label,predicate,object_label
0,glass matrix ceramic dental restoration material,rdfs:subClassOf,ceramic dental restoration material
1,polycrystalline ceramic,rdfs:subClassOf,ceramic dental restoration material
2,resin matrix ceramic,rdfs:subClassOf,ceramic dental restoration material


Looking at ontology, the above result in not complete. There more subclasses/descendants of `ceramic dental restoration material`. For example, the class `resin matrix ceramic` has subclasses:
```
resin matrix ceramic
- glass ceramic in a resin interpenetrating matrix
- resin nano ceramic
- zirconia silica ceramic in a resin interpenetrating matrix
```
To get a more complete result you need to recurse down the `ceramic dental restoration material` branch of the ontology (below).

---

## Recursively searching table

This finds all teeth, but also also returns all classifications of teeth; e.g., anterior tooth. If you want to only see specific tooth types (e.g., tooth_1), you have to return only the leaf nodes of the tooth branch (see below).

In [13]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

# recursively find descendants of class
ChildOf(x, y) :- statements(subject: x, predicate: "rdfs:subClassOf", object: y);
DescendantOf(x, y) :- ChildOf(x, y);
DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# function to return the label of a class
Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);

Material(subject:, label:) distinct :- 
    statements(subject:, object:), 
    DescendantOf(subject, "OHD:0000135"), 
    label == Label(subject);

""", 'Material', cn)

,subject,label
0,OHD:0001032,glass matrix ceramic dental restoration material
1,OHD:0001033,polycrystalline ceramic
2,OHD:0001034,zirconia polycrystalline ceramic
3,OHD:0001035,3Y-TZP zirconia ceramic
4,OHD:0001036,4Y-PSZ zirconia ceramic
5,OHD:0001037,≥ 5Y-PSZ zirconia ceramic
6,OHD:0001038,multichromatic zirconia ceramic
7,OHD:0001039,multilayered zirconia ceramic
8,OHD:0001040,polycrystalline composites ceramic
9,OHD:0001041,zirconia toughened alumina ceramic


When using the `sqlite` engine, finding only leaf nodes doesn't seem possible. Conceptually, you want all classes with `SubclassCount(x) < 1`. However, `SubclassCount(x) < 1` does not return any classes.  The `sqlite` engine does not seem to be able return a count of `0`.

In [14]:
# save query to string
query = """
@Engine("sqlite");

# recursively find descendants of class
ChildOf(x, y) :- statements(subject: x, predicate: "rdfs:subClassOf", object: y);
DescendantOf(x, y) :- ChildOf(x, y);
DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# function to return the label of a class
Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);

# queries to get list and number of descendants
Subclasses(x) List= subject :-
    statements(subject:, predicate: "rdfs:subClassOf", object:), object == x, subject != object;
SubclassCount(x) = Size(Subclasses(x));

Material(subject:, label:, x:) distinct :- 
    statements(subject:, object:), 
    DescendantOf(subject, "OHD:0000135"), 
    label == Label(subject),
    x = SubclassCount(subject),
    x {} 1;
""" 

set `x < 1` (i.e., `SubclassCount(x) < 1`) and execute query

In [15]:
logica_lib.RunPredicateFromString(query.format("<"), 'Material', cn)

,subject,label,x


set `x > 1` (i.e., `SubclassCount(x) > 1`) and execute query

In [16]:
logica_lib.RunPredicateFromString(query.format(">"), 'Material', cn)

,subject,label,x
0,OHD:0001032,glass matrix ceramic dental restoration material,2
1,OHD:0001033,polycrystalline ceramic,2
2,OHD:0001034,zirconia polycrystalline ceramic,5
3,OHD:0001040,polycrystalline composites ceramic,2
4,OHD:0001047,resin matrix ceramic,3
5,OHD:0001052,synthetic glass matrix ceramic,3


Another approach would be to test if the class has child. Leaf nodes would be those without a child. However, the statement `~HasChild(x)` produces the error:
```python
OperationalError: no such function: MagicalEntangle
```
See issue [no such function: MagicalEntangle](https://github.com/EvgSkv/logica/issues/361) for discussion.  
Uncomment and the block below to see error.

In [17]:
# # save query to string
# logica_lib.RunPredicateFromString("""
# @Engine("sqlite");

# # recursively find descendants of class
# ChildOf(x, y) :- statements(subject: x, predicate: "rdfs:subClassOf", object: y);
# DescendantOf(x, y) :- ChildOf(x, y);
# DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# # test if child has a child.
# HasChild(x) :- ChildOf(y, x);

# # function to return the label of a class
# Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);

# # queries to get list and number of descendants
# Subclasses(x) List= subject :-
#     statements(subject:, predicate: "rdfs:subClassOf", object:), object == x, subject != object;
# SubclassCount(x) = Size(Subclasses(x));

# Material(subject:) distinct :- 
#     statements(subject:, object:), 
#     DescendantOf(subject, "OHD:0000135"), 
#     ~HasChild(subject); # <--------- produces MagicalEntangle error
# """, 'Material', cn)

Oddly, if I manually define facts about materials, I can then find leaf nodes using `~HasChild(x)`. I do not know why this is.    
In this contrived example, `resin nano ceramic` is the only class that doesn't have a child. 

In [18]:
# save query to string
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

# define facts
Material(subject: "ceramic", predicate: "type", object: "material");
Material(subject: "resin", predicate: "type", object: "ceramic");
Material(subject: "resin nano ceramic", predicate: "type", object: "resin");

# recursively find descendants of class
ChildOf(x, y) :- Material(subject: x, predicate: "type", object: y);
DescendantOf(x, y) :- ChildOf(x, y);
DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# test if child has a child.
HasChild(x) :- ChildOf(y, x);

# function to return the label of a class
Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);

Materials(subject:) distinct :- 
    Material(subject:, object:), 
    DescendantOf(subject, "material"), 
    ~HasChild(subject); # <--------- does NOT produce MagicalEntangle error
""", 'Materials')

,subject
0,resin nano ceramic


In [19]:
logica_lib.RunPredicateFromString("""
@Engine("sqlite");

# define Tooth facts
Tooth(subject: "tooth", predicate: "type", object: "tooth");
Tooth(subject: "canine", predicate: "type", object: "tooth");
Tooth(subject: "upper-left canine", predicate: "type", object: "canine");
Tooth(subject: "upper-right canine", predicate: "type", object: "canine");
Tooth(subject: "bottom-left canine", predicate: "type", object: "canine");
Tooth(subject: "bottom-right canine", predicate: "type", object: "canine");

# recursively find descendants of class
ChildOf(x, y) :- Tooth(subject: x, predicate: "type", object: y);
DescendantOf(x, y) :- ChildOf(x, y);
DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# test if class is a tooth
IsaTooth(x) = y :- y == "tooth", DescendantOf(x, y);

# test if child has a child.
HasChild(x) :- ChildOf(y, x);

Teeth(tooth: subject) distinct :- 
    Tooth(subject:, predicate:, object:),
    IsaTooth(subject), 
    ~HasChild(subject); # <------ does not produce MagicalEntangle error
""", 'Teeth')

,tooth
0,bottom-left canine
1,bottom-right canine
2,upper-left canine
3,upper-right canine


---

## Working with pandas

You can query `pandas` dataframes in `logica`, but you have to use the `duckdb` engine. After creating the dataframe, you add the dataframe to `logica_lib` module using the `logica_lib.<variable> = <dataframe>`.

In [20]:
xs = np.linspace(1.0, 6.0, 100000000)
ys = np.sin(xs)
sin_wave = pd.DataFrame({'x': xs, 'y': ys})
logica_lib.sin_wave = sin_wave

In [21]:
logica_lib.RunPredicateFromString("""
@Engine("duckdb"); # <--- use duckdb
Pi() Min= x :- sin_wave(x:, y:), y < 0;
""", 'Pi', connection=duckdb.connect())

,logica_value
0,3.141593


Using `duckdb` allows you to use negation with producing the `MagicalEntangle` above. So, we can find all the leaf nodes of the tooth branch in the on ontology.

create dataframe from the statements table

In [22]:
statements_df = pd.read_sql_query("select * from statements", cn)
statements_df.head()

,stanza,subject,predicate,object,value,datatype,language
0,obo:ohd/components/dental_material.owl,obo:ohd/components/dental_material.owl,rdf:type,owl:Ontology,None,None,None
1,IAO:0000115,IAO:0000115,rdfs:label,None,definition,None,None
2,IAO:0000115,IAO:0000115,rdfs:isDefinedBy,obo:ohd/imports/envo_import.owl,None,None,None
3,IAO:0000115,IAO:0000115,rdfs:isDefinedBy,obo:ohd/imports/chebi_import.owl,None,None,None
4,IAO:0000115,IAO:0000115,rdf:type,owl:AnnotationProperty,None,None,None


assign `statements_df` to the `logica_lib.statements` variable 

In [23]:
logica_lib.statements = statements_df

find leaf nodes using `~HasChild(x)`

In [24]:
# save query to string
logica_lib.RunPredicateFromString("""
@Engine("duckdb"); # <------- using duckdb engine

# recursively find descendants of class
ChildOf(x, y) :- statements(subject: x, predicate: "rdfs:subClassOf", object: y);
DescendantOf(x, y) :- ChildOf(x, y);
DescendantOf(x, z) :- ChildOf(x, y), DescendantOf(y, z);

# test if child has a child.
HasChild(x) :- ChildOf(y, x);

# function to return the label of a class
Label(x) = value :- statements(subject: x, predicate: "rdfs:label", value:);

# queries to get list and number of descendants
Subclasses(x) List= subject :-
    statements(subject:, predicate: "rdfs:subClassOf", object:), object == x, subject != object;
SubclassCount(x) = Size(Subclasses(x));

Material(subject:, label:) distinct :- 
    statements(subject:, object:), 
    DescendantOf(subject, "OHD:0000135"), 
    label == Label(subject), 
    ~HasChild(subject); # <------- filters results to classes that don't have children
""", 'Material', connection=duckdb.connect())

,subject,label
0,OHD:0001051,feldspathic glass matrix ceramic
1,OHD:0001038,multichromatic zirconia ceramic
2,OHD:0001035,3Y-TZP zirconia ceramic
3,OHD:0001055,fluorapatite based synthetic galss matrix ceramic
4,OHD:0001050,zirconia silica ceramic in a resin interpenetr...
5,OHD:0001053,leucite based synthetic galss matrix ceramic
6,OHD:0001048,resin nano ceramic
7,OHD:0001054,lithia based synthetic galss matrix ceramic
8,OHD:0001036,4Y-PSZ zirconia ceramic
9,OHD:0001037,≥ 5Y-PSZ zirconia ceramic
